In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import random

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from functools import reduce

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [147]:
doc_to_title = {}
with open('data/docs_titles.tsv/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
#         print(line)
        data = line.strip().split('\t', 1)
#         print('data = ',data)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
# print(doc_to_title)
print (len(doc_to_title)) 
# doc_to_title #словарь id документа: заголовок

28026


In [148]:
import pandas as pd
train_data = pd.read_csv('data/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
# traingroups_titledata #Словарь номер группы: список пар (Id_doc, заголовок, target) для заголовков этой группы

In [6]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train, y_train, groups_train, X_train.shape)

[[4 4 3 ... 2 1 1]
 [8 7 7 ... 2 2 2]
 [2 2 2 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [6 2 1 ... 1 1 1]
 [3 2 2 ... 1 1 1]] [0 0 0 ... 0 0 0] [  1   1   1 ... 129 129 129] (11690, 15)


In [7]:
test_data = pd.read_csv('data/test_groups.csv')
testgroups_titledata = {}

for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [8]:
X_test = []
groups_test = []

for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:15])
X_test = np.array(X_test)
groups_test = np.array(groups_test)
X_test

array([[ 3,  3,  2, ...,  1,  1,  1],
       [ 5,  3,  3, ...,  2,  2,  2],
       [ 6,  5,  5, ...,  4,  4,  4],
       ...,
       [ 5,  4,  4, ...,  2,  2,  2],
       [ 3,  3,  3, ...,  2,  2,  2],
       [14, 14, 14, ...,  2,  2,  2]])

In [9]:
log_reg = SGDClassifier(loss='log')

In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
log_reg.fit(scaler.transform(X_train), y_train)
y_pred = log_reg.predict(scaler.transform(X_test))

In [11]:
data = test_data
data['target'] = y_pred
data = data.drop(['group_id', 'doc_id'], axis=1)

# print(data.head(100))
# data['target'] = data['target'].apply(lambda x: 0 if x == -1 else 1)
data.to_csv("submission.csv", index=False)
data

,pair_id,target
0,11691,0
1,11692,0
2,11693,1
3,11694,1
4,11695,0
...,...,...
16622,28313,0
16623,28314,0
16624,28315,1
16625,28316,0


In [12]:
np.unique(data['target'], return_counts=True)

(array([0, 1]), array([12531,  4096]))

In [13]:
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score

In [14]:
def flatten(x):
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str):
            result.extend(flatten(elem))
        else:
            result.append(elem)
    return result

In [15]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [30]:
def combinations(params):
    
    list_ = []
    for value in params.values():
        list_.append(value)

    tmp = reduce(lambda x, y: list_concat(x, y), list_)

    res = []
    for elem in tmp:
        param_list = flatten(elem)
        param_dict = dict(zip(params.keys(), param_list))
        res.append(param_dict)
        
    return res

In [31]:
combinations(params)

[{'loss': 'log', 'alpha': 0.01, 'test': 1},
 {'loss': 'log', 'alpha': 0.01, 'test': 2},
 {'loss': 'log', 'alpha': 0.01, 'test': 3},
 {'loss': 'log', 'alpha': 0.001, 'test': 1},
 {'loss': 'log', 'alpha': 0.001, 'test': 2},
 {'loss': 'log', 'alpha': 0.001, 'test': 3},
 {'loss': 'log', 'alpha': 0.0001, 'test': 1},
 {'loss': 'log', 'alpha': 0.0001, 'test': 2},
 {'loss': 'log', 'alpha': 0.0001, 'test': 3},
 {'loss': 'huber', 'alpha': 0.01, 'test': 1},
 {'loss': 'huber', 'alpha': 0.01, 'test': 2},
 {'loss': 'huber', 'alpha': 0.01, 'test': 3},
 {'loss': 'huber', 'alpha': 0.001, 'test': 1},
 {'loss': 'huber', 'alpha': 0.001, 'test': 2},
 {'loss': 'huber', 'alpha': 0.001, 'test': 3},
 {'loss': 'huber', 'alpha': 0.0001, 'test': 1},
 {'loss': 'huber', 'alpha': 0.0001, 'test': 2},
 {'loss': 'huber', 'alpha': 0.0001, 'test': 3}]

In [33]:
def group_fold_generator(X_train, train_target, groups_train, folds_num=10):
    
    groups_num = np.unique(groups_train).shape[0]
    
    for i in range(folds_num):
        
        a = [random.randint(1, groups_num) for i in range(groups_num // folds_num)]

        test_index = [i for i, x in enumerate(groups_train) if x in a]
        train_index = [i for i, x in enumerate(groups_train) if x not in a]

        yield train_index, test_index

In [45]:
testx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
testy = [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]
testg = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8]

In [46]:
for i, x in group_fold_generator(testx, testy, testg, folds_num=3):
    print(i, x)

[0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15] [6, 7]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15] [0, 1, 12, 13]
[2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15] [0, 1, 8, 9]


In [47]:
def default_fold_generator(X_train, train_target, folds_num=10, shuffle=False):
    
    folds = KFold(folds_num, shuffle = shuffle)
    
    return folds.split(X_train, train_target)

In [141]:
def validation(X_train, train_target, model, params, folds_gen_func=default_fold_generator, **kwargs):
    
    scaler = StandardScaler()
    scaler.fit(X_train) 
    
    main_res = []
    for param_set in combinations(params):
        
        print(param_set)        
        exact_model = model(**param_set) 
        
        fold_generator = folds_gen_func(X_train, train_target, **kwargs)
        
        res = []
        for train_index, test_index in fold_generator:

            exact_model.fit(scaler.transform(X_train[train_index]), train_target[train_index])
            res.append(metrics.roc_auc_score(train_target[test_index],\
                                      exact_model.predict(scaler.transform(X_train[test_index]))))
            
        mean = sum(res)/len(res)
        print(mean)
        main_res.append((mean, param_set))
    
    best = main_res[np.argmax([res[0] for res in main_res])]
    print('--------max-------')
    print(best)
    
    return best

In [142]:
params = {'loss': ['log', 'huber'],
          'alpha': [0.01, 0.001, 0.0001]}

In [143]:
combinations(params)

[{'loss': 'log', 'alpha': 0.01},
 {'loss': 'log', 'alpha': 0.001},
 {'loss': 'log', 'alpha': 0.0001},
 {'loss': 'huber', 'alpha': 0.01},
 {'loss': 'huber', 'alpha': 0.001},
 {'loss': 'huber', 'alpha': 0.0001}]

In [144]:
validation(X_train, y_train, SGDClassifier, params, group_fold_generator, groups_train=groups_train, folds_num=10)

{'loss': 'log', 'alpha': 0.01}
0.6773975755100343
{'loss': 'log', 'alpha': 0.001}
0.6821486785321039
{'loss': 'log', 'alpha': 0.0001}
0.6652391027394283
{'loss': 'huber', 'alpha': 0.01}
0.5875335507032415
{'loss': 'huber', 'alpha': 0.001}
0.5958076774134982
{'loss': 'huber', 'alpha': 0.0001}
0.6104735350284446
--------max-------
(0.6821486785321039, {'loss': 'log', 'alpha': 0.001})


(0.6821486785321039, {'loss': 'log', 'alpha': 0.001})

In [145]:
params = {'algorithm': ['ball_tree', 'kd_tree', 'brute'],
          'n_neighbors': [3, 4, 5, 6, 7, 8, 9],
          'weights': ['uniform', 'distance']}

In [146]:
validation(X_train, y_train, KNeighborsClassifier, params, group_fold_generator, groups_train=groups_train, folds_num=43)

{'algorithm': 'ball_tree', 'n_neighbors': 3, 'weights': 'uniform'}
0.6343889493876655
{'algorithm': 'ball_tree', 'n_neighbors': 3, 'weights': 'distance'}
0.6004258645162373
{'algorithm': 'ball_tree', 'n_neighbors': 4, 'weights': 'uniform'}
0.6127212696295192
{'algorithm': 'ball_tree', 'n_neighbors': 4, 'weights': 'distance'}
0.6446129412342354
{'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'uniform'}
0.631803203683431
{'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'distance'}
0.645270681270512
{'algorithm': 'ball_tree', 'n_neighbors': 6, 'weights': 'uniform'}
0.6462149864343416
{'algorithm': 'ball_tree', 'n_neighbors': 6, 'weights': 'distance'}
0.6281836783625667
{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'uniform'}
0.6368612454738048
{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}
0.6490938664481747
{'algorithm': 'ball_tree', 'n_neighbors': 8, 'weights': 'uniform'}
0.6278316202928559
{'algorithm': 'ball_tree', 'n_neighbors': 8, 'weigh

(0.6730239511912788,
 {'algorithm': 'brute', 'n_neighbors': 9, 'weights': 'distance'})

In [64]:
scaler = StandardScaler()
scaler.fit(X_train) 
clf = KNeighborsClassifier(algorithm='brute', n_neighbors=7, weights='distance')
print(clf.fit(scaler.transform(X_train), y_train))

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='distance')


In [65]:
y_pred = clf.predict(scaler.transform(X_test))
print(y_pred)

[0 1 0 ... 0 1 1]


In [66]:
data = test_data
data['target'] = y_pred
data = data.drop(['group_id', 'doc_id'], axis=1)

# print(data.head(100))
# data['target'] = data['target'].apply(lambda x: 0 if x == -1 else 1)
data.to_csv("submission.csv", index=False)
data

,pair_id,target
0,11691,0
1,11692,1
2,11693,0
3,11694,1
4,11695,0
...,...,...
16622,28313,0
16623,28314,0
16624,28315,0
16625,28316,1


In [67]:
np.unique(data['target'], return_counts=True)

(array([0, 1]), array([12886,  3741]))

In [68]:
params = {'n_estimators': [100, 200, 500, 1000],
          'criterion': ['gini', 'entropy'],
          'max_depth': [10, 15, None],
          'n_jobs': [-1]}

In [69]:
validation(X_train, y_train, RandomForestClassifier, params, group_fold_generator, groups_train=groups_train, folds_num=10)

{'n_estimators': 100, 'criterion': 'gini', 'max_depth': 10, 'n_jobs': -1}
0.6704299199571107
{'n_estimators': 100, 'criterion': 'gini', 'max_depth': 15, 'n_jobs': -1}
0.657575455696904
{'n_estimators': 100, 'criterion': 'gini', 'max_depth': None, 'n_jobs': -1}
0.6532749827169279
{'n_estimators': 100, 'criterion': 'entropy', 'max_depth': 10, 'n_jobs': -1}
0.6839848103541722
{'n_estimators': 100, 'criterion': 'entropy', 'max_depth': 15, 'n_jobs': -1}
0.7001529039815791
{'n_estimators': 100, 'criterion': 'entropy', 'max_depth': None, 'n_jobs': -1}
0.6489330733847145
{'n_estimators': 200, 'criterion': 'gini', 'max_depth': 10, 'n_jobs': -1}
0.653281615873575
{'n_estimators': 200, 'criterion': 'gini', 'max_depth': 15, 'n_jobs': -1}
0.676515027188833
{'n_estimators': 200, 'criterion': 'gini', 'max_depth': None, 'n_jobs': -1}
0.6539468724973003
{'n_estimators': 200, 'criterion': 'entropy', 'max_depth': 10, 'n_jobs': -1}
0.6735844696499337
{'n_estimators': 200, 'criterion': 'entropy', 'max_dept

In [70]:
clf = RandomForestClassifier(n_estimators=500, criterion='entropy')
print(clf.fit(X_train, y_train))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [71]:
y_pred = clf.predict(X_test)
print(y_pred)

[0 1 0 ... 0 0 1]


In [72]:
data = test_data
data['target'] = y_pred
data = data.drop(['group_id', 'doc_id'], axis=1)

# print(data.head(100))
# data['target'] = data['target'].apply(lambda x: 0 if x == -1 else 1)
data.to_csv("submission.csv", index=False)
data

,pair_id,target
0,11691,0
1,11692,1
2,11693,0
3,11694,1
4,11695,0
...,...,...
16622,28313,0
16623,28314,0
16624,28315,0
16625,28316,0


In [73]:
np.unique(data['target'], return_counts=True)

(array([0, 1]), array([13025,  3602]))